# Example: Cats vs. Dogs With SqueezeNet

This notebook demonstrates the usage of ``image_featurizer`` using the Kaggle Cats vs. Dogs dataset.

We will look at the usage of the ``ImageFeaturizer()`` class, which provides a convenient pipeline to quickly tackle image problems with DataRobot's platform. 

It allows users to load image data into the featurizer, and then featurizes the images into a maximum of 2048 features. It appends these features to the CSV as extra columns in line with the image rows. If no CSV was passed in with an image directory, the featurizer generates a new CSV automatically and performs the same function.


In [1]:
# Setting up stdout logging for the example case
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

In [2]:
# Importing the dependencies for this example
import pandas as pd
import numpy as np
from sklearn import svm
from pic2vec import ImageFeaturizer


Using TensorFlow backend.


## Formatting the Data

'ImageFeaturizer' accepts as input either:
1. An image directory
2. A CSV with URL pointers to image downloads, or 
3. A combined image directory + CSV with pointers to the included images. 

For this example, we will load in the Kaggle Cats vs. Dogs dataset of 25,000 images, along with a CSV that includes each images class label.

In [3]:
pd.options.display.max_rows = 10

image_path = 'cat_vs_dogs_images/'
csv_path = 'cat_vs_dog_classes.csv'

pd.read_csv(csv_path)

,images,animal
0,cat.0.jpg,0
1,cat.1.jpg,0
2,cat.10.jpg,0
3,cat.100.jpg,0
4,cat.1000.jpg,0
...,...,...
24995,dog.9995.jpg,1
24996,dog.9996.jpg,1
24997,dog.9997.jpg,1
24998,dog.9998.jpg,1


The image directory contains 12,500 images of cats and 12,500 images of dogs. The CSV contains pointers to each image in the directory, along with a class label (0 for cats, 1 for dogs).

## Initializing the Featurizer

We will now initialize the ImageFeaturizer( ) class with a few parameters that define the model. If in doubt, we can always call the featurizer with no parameters, and it will initialize itself to a cookie-cutter build. Here, we will call the parameters explicitly to demonstrate functionality. However, these are generally the default weights, so for this build we could just call ```featurizer = ImageFeaturizer()```.

Because we have not specified a model, the featurizer will default to the built-in SqueezeNet model, with loaded weights prepackaged. If you initialize another model, pic2vec will automatically download the model weights through the Keras backend.

The depth indicates how far down we should cut the model to draw abstract features– the further down we cut, the less complex the representations will be, but they may also be less specialized to the specific classes in the ImageNet dataset that the model was trained on– and so they may perform better on data that is further from the classes within the dataset.

Automatic downsampling means that this model will downsample the final layer from 512 features to 256 features, which is a more compact representation. With large datasets and bigger models (such as InceptionV3, more features may run into memory problems or difficulty optimizing, so it may be worth downsampling to a smaller featurspace.

In [4]:
featurizer = ImageFeaturizer(depth=1, auto_sample = False, model='squeezenet')

INFO - 
Building the featurizer.
INFO - Check/download SqueezeNet model weights. This may take a minute first time.
INFO - Model successfully initialized.
INFO - Model decapitated.
INFO - Model downsampled.
INFO - Full featurizer is built.
INFO - No downsampling. Final layer feature space has size 512


This featurizer was 'decapitated' to the first layer below the prediction layer, which will produce complex representations. Because it is so close to the final prediction layer, it will create more specialized feature representations, and therefore will be better suited for image datasets that are similar to classes within the original ImageNet dataset. Cats and dogs are present within ImageNet, so a depth of 1 should perform well. 

## Loading the Data

Now that the featurizer is built, we can load our data into the network. This will parse through the images in the order given by the csv, rescale them to a target size depending on the network– SqueezeNet is (227, 227)– and build a 4D tensor containing the vectorized representations of the images. This tensor will later be fed into the network in order to be featurized.

The tensor will have the dimensions: [number of images, height, width, color channels]. In this case, the image tensor will have size [25000, 227, 227, 3].

We have to pass in the name of the column in the CSV that contains pointers to the images, as well as the path to the image directory and the path to the CSV itself, which are both saved from earlier. 

If there are images in the directory that aren't in the CSV, or image names in the CSV that aren't in the directory, or even files that aren't valid image files in the directory, don't fear– the featurizer will only try to vectorize valid images that are present in both the CSV and the directory. Any images present in the CSV but not the directory will be given zero vectors, and the order of the CSV is considered the canonical order for the images.

In [5]:
featurizer.load_data('images', image_path = image_path, csv_path = csv_path)

INFO - Found image paths that overlap between both the directory and the csv.
INFO - Converting images.
INFO - Converted 0 images. Only 25000 images left to go.
INFO - Converted 1000 images. Only 24000 images left to go.
INFO - Converted 2000 images. Only 23000 images left to go.
INFO - Converted 3000 images. Only 22000 images left to go.
INFO - Converted 4000 images. Only 21000 images left to go.
INFO - Converted 5000 images. Only 20000 images left to go.
INFO - Converted 6000 images. Only 19000 images left to go.
INFO - Converted 7000 images. Only 18000 images left to go.
INFO - Converted 8000 images. Only 17000 images left to go.
INFO - Converted 9000 images. Only 16000 images left to go.
INFO - Converted 10000 images. Only 15000 images left to go.
INFO - Converted 11000 images. Only 14000 images left to go.
INFO - Converted 12000 images. Only 13000 images left to go.
INFO - Converted 13000 images. Only 12000 images left to go.
INFO - Converted 14000 images. Only 11000 images left t

The image data has now been loaded into the featurizer and vectorized, and is ready for featurization. We can check the size, format, and other stored information about the data by calling the featurizer object attributes:

In [6]:
print('Vectorized data shape: {}'.format(featurizer.data.shape))

print('CSV path: \'{}\''.format(featurizer.csv_path))

print('Image directory path: \'{}\''.format(featurizer.image_path))

Vectorized data shape: (25000, 227, 227, 3)
CSV path: 'cat_vs_dog_classes.csv'
Image directory path: 'cat_vs_dogs_images/'


For a full list of attributes, call:

In [7]:
featurizer.__dict__.keys()

['downsample_size',
 'visualize',
 'auto_sample',
 'image_column_header',
 'isotropic_scaling',
 'csv_path',
 'number_crops',
 'image_list',
 'featurizer',
 'scaled_size',
 'depth',
 'featurized_data',
 'crop_size',
 'data',
 'model_name',
 'image_path']

## Featurizing the Data

Now that the data is loaded, we're ready to featurize the data. This will push the vectorized images through the network and save the 2D matrix output– each row representing a single image, and each column storing a different feature.

It will then create and save a new CSV by appending these features to the end of the given CSV in line with each image's row. The features themselves will also be saved in a separate CSV file without the image names or other data. Both generated CSVs will be saved to the same path as the original CSV, with the features-only CSV appending '_features_only' and the combined CSV appending '_full' to the end of their respective filenames.

The featurize( ) method requires no parameters, as it uses the data we just loaded into the network. This requires pushing images through the deep network, and so if you choose to use a slower, more powerful model like InceptionV3, relatively large datasets will require a GPU to perform in a reasonable amount of time. Using a mid-range GPU, it can take about 30 minutes to process the full 25,000 photos in the Dogs vs. Cats through InceptionV3. On the other hand, if you would like a fast model, lightweight model without top-of-the-line accuracy, SqueezeNet works well enough and can perform inference on CPUs quickly.

In [8]:
featurizer.featurize()

INFO - Checking array initialized.
INFO - Trying to featurize data.
INFO - Creating feature array.
25000/25000 [==============================] - 130s   
INFO - Feature array created successfully.
INFO - Adding image features to csv.


,images,animal,image_feature_0,image_feature_1,image_feature_2,image_feature_3,image_feature_4,image_feature_5,image_feature_6,image_feature_7,...,image_feature_502,image_feature_503,image_feature_504,image_feature_505,image_feature_506,image_feature_507,image_feature_508,image_feature_509,image_feature_510,image_feature_511
0,cat.0.jpg,0,0.000000,0.000000,0.000000,0.976163,0.000000,0.0,0.0,0.000000,...,0.326453,0.125493,0.0,0.000000,20.787928,0.000000,0.827707,0.072728,0.123821,0.361559
1,cat.1.jpg,0,0.020484,0.000000,0.000000,1.537819,0.000000,0.0,0.0,0.000000,...,0.626090,0.132664,0.0,0.000000,19.593006,0.000914,1.273858,0.112284,0.076074,0.626272
2,cat.10.jpg,0,0.000000,0.000000,0.002142,3.562325,0.000000,0.0,0.0,0.000000,...,0.558218,0.247748,0.0,0.000000,15.065393,0.004629,1.506223,0.357541,0.225619,0.313667
3,cat.100.jpg,0,0.000000,0.003331,0.000000,2.565698,0.003193,0.0,0.0,0.000356,...,1.230276,0.587366,0.0,0.000000,16.498240,0.000000,0.457677,0.245788,0.243526,0.209688
4,cat.1000.jpg,0,0.000000,0.000000,0.000000,0.687325,0.166234,0.0,0.0,0.000000,...,0.270655,0.075184,0.0,0.000000,17.514622,0.000000,0.854873,0.008352,0.016658,0.344797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,dog.9995.jpg,1,0.000000,0.000000,0.000000,1.193059,0.000000,0.0,0.0,0.000000,...,0.268092,0.177756,0.0,0.000000,20.336287,0.000000,0.896977,0.149458,0.123787,0.384676
24996,dog.9996.jpg,1,0.000000,0.003674,0.000000,2.768917,0.000000,0.0,0.0,0.000000,...,0.777494,0.171866,0.0,0.001702,17.666206,0.000000,0.779997,0.034684,0.133014,1.652294
24997,dog.9997.jpg,1,0.000000,0.000000,0.000000,1.627975,0.000000,0.0,0.0,0.000000,...,0.797773,0.114264,0.0,0.000000,20.120745,0.000000,0.980927,0.005060,0.006118,0.623514
24998,dog.9998.jpg,1,0.000000,0.044554,0.000000,1.563408,0.000000,0.0,0.0,0.000000,...,0.939854,0.059493,0.0,0.000000,18.924351,0.000000,1.038141,0.069819,0.069094,0.904590


## Results

The dataset has now been fully featurized! The features are saved under the featurized_data attribute:

In [9]:
featurizer.featurized_data

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.27284253e-02,   1.23820536e-01,   3.61558944e-01],
       [  2.04837546e-02,   0.00000000e+00,   0.00000000e+00, ...,
          1.12284146e-01,   7.60741457e-02,   6.26272500e-01],
       [  0.00000000e+00,   0.00000000e+00,   2.14172155e-03, ...,
          3.57541353e-01,   2.25618958e-01,   3.13666701e-01],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          5.05957752e-03,   6.11788966e-03,   6.23513699e-01],
       [  0.00000000e+00,   4.45538573e-02,   0.00000000e+00, ...,
          6.98191002e-02,   6.90938309e-02,   9.04590428e-01],
       [  0.00000000e+00,   0.00000000e+00,   1.77303536e-05, ...,
          1.47210872e-02,   1.11560710e-01,   9.44222987e-01]], dtype=float32)

The full data has also been successfully saved in CSV form, which allows it to be dropped directly into the DataRobot app:

In [10]:
pd.read_csv('cat_vs_dog_classes_full.csv')

,images,animal,image_feature_0,image_feature_1,image_feature_2,image_feature_3,image_feature_4,image_feature_5,image_feature_6,image_feature_7,...,image_feature_502,image_feature_503,image_feature_504,image_feature_505,image_feature_506,image_feature_507,image_feature_508,image_feature_509,image_feature_510,image_feature_511
0,cat.0.jpg,0,0.000000,0.000000,0.000000,0.976163,0.000000,0.0,0.0,0.000000,...,0.326453,0.125493,0.0,0.000000,20.787928,0.000000,0.827707,0.072728,0.123821,0.361559
1,cat.1.jpg,0,0.020484,0.000000,0.000000,1.537819,0.000000,0.0,0.0,0.000000,...,0.626090,0.132664,0.0,0.000000,19.593006,0.000914,1.273858,0.112284,0.076074,0.626272
2,cat.10.jpg,0,0.000000,0.000000,0.002142,3.562325,0.000000,0.0,0.0,0.000000,...,0.558218,0.247748,0.0,0.000000,15.065393,0.004629,1.506223,0.357541,0.225619,0.313667
3,cat.100.jpg,0,0.000000,0.003331,0.000000,2.565698,0.003193,0.0,0.0,0.000356,...,1.230276,0.587366,0.0,0.000000,16.498240,0.000000,0.457677,0.245788,0.243526,0.209688
4,cat.1000.jpg,0,0.000000,0.000000,0.000000,0.687325,0.166234,0.0,0.0,0.000000,...,0.270655,0.075184,0.0,0.000000,17.514622,0.000000,0.854873,0.008352,0.016658,0.344797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,dog.9995.jpg,1,0.000000,0.000000,0.000000,1.193059,0.000000,0.0,0.0,0.000000,...,0.268092,0.177756,0.0,0.000000,20.336287,0.000000,0.896977,0.149458,0.123787,0.384676
24996,dog.9996.jpg,1,0.000000,0.003674,0.000000,2.768917,0.000000,0.0,0.0,0.000000,...,0.777494,0.171866,0.0,0.001702,17.666206,0.000000,0.779997,0.034684,0.133014,1.652294
24997,dog.9997.jpg,1,0.000000,0.000000,0.000000,1.627975,0.000000,0.0,0.0,0.000000,...,0.797773,0.114264,0.0,0.000000,20.120745,0.000000,0.980927,0.005060,0.006118,0.623514
24998,dog.9998.jpg,1,0.000000,0.044554,0.000000,1.563408,0.000000,0.0,0.0,0.000000,...,0.939854,0.059493,0.0,0.000000,18.924351,0.000000,1.038141,0.069819,0.069094,0.904590


But, for the purposes of this demo, we can simply test the performance of a linear classifier over the featurized data. First, we'll build the training and test sets. 

In [11]:
# Creating a training set of 10,000 for each class
train_cats = featurizer.featurized_data[:10000, :]
train_dogs = featurizer.featurized_data[12500:22500, :]

# Creating a test set from the remaining 2,500 of each class
test_cats = featurizer.featurized_data[10000:12500, :]
test_dogs = featurizer.featurized_data[22500:, :]

# Combining the training data, and creating the class labels
train_combined = np.concatenate((train_cats, train_dogs))
labels_train = np.concatenate((np.zeros((10000,)), np.ones((10000,))))

# Combining the test data, and creating the class labels to check predictions
test_combined = np.concatenate((test_cats, test_dogs))
labels_test = np.concatenate((np.zeros((2500,)), np.ones((2500,))))

Then, we'll train the linear classifier:

In [12]:
# Initialize the linear SVC
clf = svm.LinearSVC()

# Fit it on the training data
clf.fit(train_combined, labels_train)

# Check the performance of the linear classifier over the full Cats vs. Dogs dataset!
clf.score(test_combined, labels_test)

0.94320000000000004

After running the Cats vs. Dogs dataset through the lightest-weight pic2vec model, we find that a simple linear classifier trained over the featurized data achieves over 94% accuracy on distinguishing dogs vs. cats out of the box.

## Summary

That's it! We've looked at the following:

1. What data formats can be passed into the featurizer
2. How to initialize a simple featurizer
3. How to load data into the featurizer
4. How to featurize the loaded data

And as a bonus, we looked at how we might use the featurized data to perform predictions without dropping the CSV into the DataRobot app.

Unless you would like to examine the loaded data before featurizing it, steps 3 and 4 can actually be combined into a single step with the load_and_featurize_data( ) method.

## Next Steps

We have not covered using only a CSV with URL pointers, or a more complex dataset. That will be the subject of another Notebook. 

To have more control over the options in the featurizer, or to understand its internal functioning more fully, check out the full package documentation.